In [8]:
from ugot import ugot
got = ugot.UGOT()
got.initialize('192.168.1.217')

# version 2 - move while holding button / key down, stop when release
# engineer vehicle edition - with arm AND joint control

import pygame
import sys

pygame.init()

WIDTH, HEIGHT = 800, 400
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Robot Control")

WHITE = (255, 255, 255)
GRAY = (200, 200, 200)
BLACK = (0, 0, 0)

BUTTON_SIZE = 80
BUTTON_GAP = 50
UP_POS = (WIDTH // 2 - BUTTON_SIZE // 2 - 200, HEIGHT // 2 - BUTTON_SIZE - BUTTON_GAP)
DOWN_POS = (WIDTH // 2 - BUTTON_SIZE // 2 - 200, HEIGHT // 2 + BUTTON_GAP)
LEFT_POS = (WIDTH // 2 - BUTTON_SIZE - BUTTON_GAP - 200, HEIGHT // 2 - BUTTON_GAP)
RIGHT_POS = (WIDTH // 2 + BUTTON_GAP - 200, HEIGHT // 2 - BUTTON_GAP)
OPEN_POS = (WIDTH // 2 - BUTTON_SIZE - BUTTON_GAP + 200, HEIGHT // 2 - BUTTON_SIZE)
CLOSE_POS = (WIDTH // 2 - BUTTON_SIZE - BUTTON_GAP + 200, HEIGHT // 2 + BUTTON_SIZE)

#############################

# Add button positions for joint control
JOINT_BUTTON_SIZE = 60
base_x = WIDTH // 2 + 200
joint_y_start = HEIGHT // 2 - 100
JOINT_GAP = 30

joint1_up_pos = (base_x, joint_y_start)
joint1_down_pos = (base_x + JOINT_BUTTON_SIZE + 10, joint_y_start)

joint2_up_pos = (base_x, joint_y_start + JOINT_BUTTON_SIZE + JOINT_GAP)
joint2_down_pos = (base_x + JOINT_BUTTON_SIZE + 10, joint_y_start + JOINT_BUTTON_SIZE + JOINT_GAP)

joint3_up_pos = (base_x, joint_y_start + 2 * (JOINT_BUTTON_SIZE + JOINT_GAP))
joint3_down_pos = (base_x + JOINT_BUTTON_SIZE + 10, joint_y_start + 2 * (JOINT_BUTTON_SIZE + JOINT_GAP))

joint1_angle = 0
joint2_angle = 0
joint3_angle = 0
DURATION = 100  # Duration for each movement

got.mechanical_joint_control(joint1_angle, joint2_angle, joint3_angle, DURATION) # reset arm - optional

def angle(value, min_val, max_val):
    return max(min_val, min(value, max_val))

def update_joint_angles(j1, j2, j3):
    global joint1_angle, joint2_angle, joint3_angle
    joint1_angle = angle(j1, -90, 90)
    joint2_angle = angle(j2, -80, 110)
    joint3_angle = angle(j3, -90, 90)
    got.mechanical_joint_control(joint1_angle, joint2_angle, joint3_angle, DURATION)

# Define increase/decrease functions for each joint
def joint1_inc():
    update_joint_angles(joint1_angle + 3, joint2_angle, joint3_angle)

def joint1_dec():
    update_joint_angles(joint1_angle - 3, joint2_angle, joint3_angle)

def joint2_inc():
    update_joint_angles(joint1_angle, joint2_angle + 3, joint3_angle)

def joint2_dec():
    update_joint_angles(joint1_angle, joint2_angle - 3, joint3_angle)

def joint3_inc():
    update_joint_angles(joint1_angle, joint2_angle, joint3_angle + 3)

def joint3_dec():
    update_joint_angles(joint1_angle, joint2_angle, joint3_angle - 3)

##########################################################

def forward():
    print("Moving forward", end="\r")
    got.mecanum_translate_speed(angle=0, speed=50)
    # got.transform_move_speed(direction=0,speed=30)

def backward():
    print("Moving backward", end="\r")
    got.mecanum_translate_speed(angle=180, speed=50)
    # got.transform_move_speed(direction=1,speed=30)

def left():
    print("Turning left", end="\r")
    got.mecanum_turn_speed(turn=2, speed=40)
    # got.transform_turn_speed(turn=2, speed=30)

def right():
    print("Turning right", end="\r")
    got.mecanum_turn_speed(turn=3, speed=40)
    # got.transform_turn_speed(turn=3, speed=30)

def open_gripper():
    print("Opening gripper", end = "\r")
    got.mechanical_clamp_release()

def close_gripper():
    print("Closing gripper", end = "\r")
    got.mechanical_clamp_close()

def draw_button(pos, text):
    rect = pygame.Rect(pos[0], pos[1], BUTTON_SIZE, BUTTON_SIZE)
    pygame.draw.rect(screen, GRAY, rect, border_radius=5)
    font = pygame.font.SysFont('Arial', 40)
    text_surf = font.render(text, True, BLACK)
    text_rect = text_surf.get_rect(center=rect.center)
    screen.blit(text_surf, text_rect)
    return rect

running = True
# add flags
moving_forward = False
moving_backward = False
turning_left = False
turning_right = False

##########################
# Joint movement flags
moving_joint1_up = False
moving_joint1_down = False
moving_joint2_up = False
moving_joint2_down = False
moving_joint3_up = False
moving_joint3_down = False
###########################

while running:
    screen.fill(WHITE)

    up_rect = draw_button(UP_POS, '↑')
    down_rect = draw_button(DOWN_POS, '↓')
    left_rect = draw_button(LEFT_POS, '←')
    right_rect = draw_button(RIGHT_POS, '→')
    open_rect = draw_button(OPEN_POS, "Open")
    close_rect = draw_button(CLOSE_POS, "Close")

    j1_up = draw_button(joint1_up_pos, "+1")
    j1_down = draw_button(joint1_down_pos, "-1")
    j2_up = draw_button(joint2_up_pos, "+2")
    j2_down = draw_button(joint2_down_pos, "-2")
    j3_up = draw_button(joint3_up_pos, "+3")
    j3_down = draw_button(joint3_down_pos, "-3")

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # instead of moving once, set flags while holding button down
        if event.type == pygame.MOUSEBUTTONDOWN:
            if up_rect.collidepoint(event.pos):
                # forward()
                moving_forward = True
            elif down_rect.collidepoint(event.pos):
                # backward()
                moving_backward = True
            elif left_rect.collidepoint(event.pos):
                # left()
                turning_left = True
            elif right_rect.collidepoint(event.pos):
                # right()
                turning_right = True
            elif open_rect.collidepoint(event.pos):
                open_gripper()
            elif close_rect.collidepoint(event.pos):
                close_gripper()
            elif j1_up.collidepoint(event.pos):
                moving_joint1_up = True
            elif j1_down.collidepoint(event.pos):
                moving_joint1_down = True
            elif j2_up.collidepoint(event.pos):
                moving_joint2_up = True
            elif j2_down.collidepoint(event.pos):
                moving_joint2_down = True
            elif j3_up.collidepoint(event.pos):
                moving_joint3_up = True
            elif j3_down.collidepoint(event.pos):
                moving_joint3_down = True

        
        # stop moving when release button
        if event.type == pygame.MOUSEBUTTONUP:
            moving_forward = False
            moving_backward = False
            turning_left = False
            turning_right = False

            moving_joint1_up = False
            moving_joint1_down = False
            moving_joint2_up = False
            moving_joint2_down = False
            moving_joint3_up = False
            moving_joint3_down = False

        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                moving_forward = True
            elif event.key == pygame.K_DOWN:
                moving_backward = True
            elif event.key == pygame.K_LEFT:
                turning_left = True
            elif event.key == pygame.K_RIGHT:
                turning_right = True
            elif event.key == pygame.K_q:
                moving_joint1_up = True
            elif event.key == pygame.K_a:
                moving_joint1_down = True
            elif event.key == pygame.K_w:
                moving_joint2_up = True
            elif event.key == pygame.K_s:
                moving_joint2_down = True
            elif event.key == pygame.K_e:
                moving_joint3_up = True
            elif event.key == pygame.K_d:
                moving_joint3_down = True

        if event.type == pygame.KEYUP:
            if event.key == pygame.K_UP:
                moving_forward = False
            elif event.key == pygame.K_DOWN:
                moving_backward = False
            elif event.key == pygame.K_LEFT:
                turning_left = False
            elif event.key == pygame.K_RIGHT:
                turning_right = False
            elif event.key == pygame.K_q:
                moving_joint1_up = False
            elif event.key == pygame.K_a:
                moving_joint1_down = False
            elif event.key == pygame.K_w:
                moving_joint2_up = False
            elif event.key == pygame.K_s:
                moving_joint2_down = False
            elif event.key == pygame.K_e:
                moving_joint3_up = False
            elif event.key == pygame.K_d:
                moving_joint3_down = False

    # Perform actions based on movement flags
    if moving_forward:
        forward()
    elif moving_backward:
        backward()
    elif turning_left:
        left()
    elif turning_right:
        right()
    else: # very important, otherwise will never stop
        got.mecanum_stop()

    if moving_joint1_up:
        joint1_inc()
    elif moving_joint1_down:
        joint1_dec()

    if moving_joint2_up:
        joint2_inc()
    elif moving_joint2_down:
        joint2_dec()

    if moving_joint3_up:
        joint3_inc()
    elif moving_joint3_down:
        joint3_dec()
    
    pygame.display.flip()

pygame.quit()
sys.exit()


192.168.1.217:50051


SystemExit: 